## Question 1: 

In [4]:
import numpy as np
import math

N = 80
pi = math.pi
TOL = 1e-5
MAX_ITER = 10000

# Helper to get matrix element a_{i,j} (i, j are 1-based)
def get_a(i, j):
    if j == i:
        return 2*i
    elif (j == i+2 and i <= N-2) or (j == i-2 and i >= 3):
        return 0.5*i
    elif (j == i+4 and i <= N-4) or (j == i-4 and i >= 5):
        return 0.25*i
    else:
        return 0.0

# jacobi
def jacobi():
    x = np.zeros(N)
    x_new = np.zeros(N)
    for it in range(MAX_ITER):
        for i in range(N):
            idx = i+1 # 1-based
            sum_ = 0.0
            for j in [idx+2, idx-2, idx+4, idx-4]:
                if 1 <= j <= N:
                    sum_ += get_a(idx, j) * x[j-1]
            x_new[i] = (pi - sum_) / get_a(idx, idx)
        if np.linalg.norm(x_new - x, ord=np.inf) < TOL:
            print(f"Jacobi iterations to react tolerance: {it+1}\n l_inf norm: {np.linalg.norm(x_new-x, ord=np.inf):.2e}")
            return x_new
        x[:] = x_new
    raise RuntimeError("Jacobi did not converge")

# gauss-seidel
def gauss_seidel():
    x = np.zeros(N)
    for it in range(MAX_ITER):
        x_old = x.copy()
        for i in range(N):
            idx = i+1 # 1-based
            sum_ = 0.0
            # Use latest values for lower indices, old for higher
            for j in [idx+2, idx-2, idx+4, idx-4]:
                if 1 <= j <= N:
                    if j-1 < i:
                        sum_ += get_a(idx, j) * x[j-1]
                    else:
                        sum_ += get_a(idx, j) * x_old[j-1]
            x[i] = (pi - sum_) / get_a(idx, idx)
        if np.linalg.norm(x - x_old, ord=np.inf) < TOL:
            print(f"Gauss Seidel iterations to reach tolerance 10^-9: {it+1}\n l_inf norm: {np.linalg.norm(x-x_old, ord=np.inf):.2e}")
            return x
    raise RuntimeError("Gauss-Seidel did not converge")

# Run both methods
x_jacobi = jacobi()
x_gs = gauss_seidel()

print("First 10 Jacobi solution values:", x_jacobi[:10])
print("First 10 Gauss-Seidel solution values:", x_gs[:10])

Jacobi iterations to react tolerance: 33
 l_inf norm: 8.89e-06
Gauss Seidel iterations to reach tolerance 10^-9: 8
 l_inf norm: 6.85e-06
First 10 Jacobi solution values: [1.53873501 0.73142167 0.10797136 0.1732853  0.04055865 0.08525019
 0.1664504  0.12198156 0.10125265 0.09045966]
First 10 Gauss-Seidel solution values: [1.5387327  0.73141966 0.10796931 0.1732834  0.04055595 0.08524787
 0.16644711 0.12197878 0.10124911 0.09045662]


## Question 2:

In [ ]:
def solve(n, alpha):
    A = np.zeros((n-1, n-1))
    b = np.zeros(n-1)
    b[0] = alpha
    # Fill A
    for i in range(n-1):
        A[i, i] = 1.0
        if i > 0:
            A[i, i-1] = -alpha
        if i < n-2:
            A[i, i+1] = -(1-alpha)
    # Solve
    P = np.linalg.solve(A, b)
    return P

for n in [10, 50, 100]:
    P = solve(n, 0.5)
    print(f"n = {n}, Solution P_1 to P_{n-1}:")
    print(P)
    print()

In [ ]:
alpha = 0.7
for n in [10, 50, 100]:
    P = solve_general_alpha(n, alpha)
    print(f"(c) n = {n}, alpha = {alpha}, Solution P_1 to P_{n-1}:")
    print(P)
    print()

# Part (d): alpha = 1/3
alpha = 1/3
print(alpha)
for n in [10, 50, 100]:
    P = solve_general_alpha(n, alpha)
    print(f"(d) n = {n}, alpha = {alpha}, Solution P_1 to P_{n-1}:")
    print(P)
    print()

(c) n = 10, alpha = 0.7, Solution P_1 to P_9:
[0.99972122 0.99907073 0.99755293 0.99401139 0.9857478  0.96646609
 0.92147543 0.81649721 0.57154805]

(c) n = 50, alpha = 0.7, Solution P_1 to P_49:
[1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         0.99999999 0.99999998 0.99999996
 0.9999999  0.99999976 0.99999944 0.9999987  0.99999698 0.99999295
 0.99998354 0.9999616  0.99991041 0.99979096 0.99951224 0.99886189
 0.9973444  0.9938036  0.98554174 0.96626406 0.9212828  0.81632653
 0.57142857]

(c) n = 100, alpha = 0.7, Solution P_1 to P_99:
[1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1

In [10]:
def hilbert(n):
    H = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            H[i, j] = 1.0 / (i + j + 1)
    return H

n = 10
H = hilbert(n)
I = np.eye(n)
A = H + 0.01 * I
x_exact = np.ones(n)
b = A @ x_exact

# Minimum-Residual (MR) method
def minres(A, b, tol=1e-9, max_iter=10000):
    x = np.zeros_like(b)
    r = b - A @ x
    r0_norm = np.linalg.norm(r)
    steps = 0
    while np.linalg.norm(r) / r0_norm > tol and steps < max_iter:
        Ar = A @ r
        alpha = np.dot(r, Ar) / np.dot(Ar, Ar)
        x = x + alpha * r
        r = b - A @ x
        steps += 1
    return x, steps

# Minimum-Residual with 1D Krylov (Gauss-Seidel direction)
def minres_gs(A, b, tol=1e-9, max_iter=10000):
    x = np.zeros_like(b)
    r = b - A @ x
    r0_norm = np.linalg.norm(r)
    steps = 0
    n = len(b)
    while np.linalg.norm(r) / r0_norm > tol and steps < max_iter:
        # Use Gauss-Seidel direction: solve Az = r by one GS sweep
        z = np.zeros_like(r)
        for i in range(n):
            sum1 = np.dot(A[i, :i], z[:i])
            sum2 = np.dot(A[i, i+1:], z[i+1:])
            z[i] = (r[i] - sum1 - sum2) / A[i, i]
        alpha = np.dot(r, A @ z) / np.dot(A @ z, A @ z)
        x = x + alpha * z
        r = b - A @ x
        steps += 1
    return x, steps

# Run both methods and compare steps
x_mr, steps_mr = minres(A, b)
x_mr_gs, steps_mr_gs = minres_gs(A, b)

print(f"Regular Minimum-Residual steps: {steps_mr}")
print(f"Minimum-Residual (GS direction) steps: {steps_mr_gs}")
print("Final error (regular):", np.linalg.norm(x_mr - x_exact))
print("Final error (GS direction):", np.linalg.norm(x_mr_gs - x_exact))

Regular Minimum-Residual steps: 955
Minimum-Residual (GS direction) steps: 133
Final error (regular): 4.5312517031532674e-07
Final error (GS direction): 2.1038024936827903e-07
